In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time
from tqdm import tqdm
import fitz

from functools import reduce

/opt/homebrew/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### importing books

In [3]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [4]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

In [5]:
#mout. index: 
vol1_index_path = '../output/local/index_output/vol1_nonitalics.csv'
vol2_index_path = '../output/local/index_output/vol2_nonitalics.csv'
vol3_index_path = '../output/local/index_output/vol3_nonitalics.csv'

vol1_index_df = pd.read_csv(vol1_index_path)
vol2_index_df = pd.read_csv(vol2_index_path)
vol3_index_df = pd.read_csv(vol3_index_path)

#changing name of columns of mout. indecies 
vol1_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol2_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol3_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)

In [6]:
vol1_word_df = vol1_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol2_word_df = vol2_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol3_word_df = vol3_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

### getting word pairs

*right now last word of page i matches with alst word of page i + 1*

things that I need to account for and am not doing so right now: 
- what if a word is in the next paragraph/block 
- what if a word is in the next page 

In [7]:
#list of genera from index -- uppercased to match main text pattern
vol1_genera = vol1_index_df[vol1_index_df['taxon_rank'] == 'genus']['mouterde_genus'].str.upper().tolist()

#list of species binomial from main text
vol1_species_temp_df = vol1_index_df[(vol1_index_df['taxon_rank'] == 'epithet') & (~vol1_index_df['mouterde_genus'].isna())]
vol1_species_binomial_list = list(zip(vol1_species_temp_df['mouterde_genus'], vol1_species_temp_df['mouterde_epithet']))
vol1_species = list(map(lambda x: f"{x[0]} {x[1]}", vol1_species_binomial_list))
vol1_species_abriviation = list(map(lambda x: f"{x[0][0]}. {x[1]}", vol1_species_binomial_list))

In [8]:
def is_italic(flags):
    return flags & 2 ** 1 != 0

In [9]:
def get_n_words_flagged(df, n, inplace = True):
    #assumes n > 1
    out_words_col = f"{n}_words"
    out_flags_col = f"{n}_flags"
    
    line_group_cols = ['vol_num', 'page_num', 
                       'block_num', 'block_num_absolute', 'block_bbox', 
                       'line_num', 'line_wmode', 'line_dir', 'line_bbox']
    
    n_words_lists = [None for i in range(n)]
    n_words_flags = [None for i in range(n)]

    n_words_lists[0] = df['word']
    n_words_flags[0] = df['span_flags']

    for i in range(1, n):
        n_words_lists[i] = df.groupby(line_group_cols)['word'].shift(-i, fill_value="")
        n_words_flags[i] = df.groupby(line_group_cols)['span_flags'].shift(-i, fill_value=0)

    zip_n_words = list(zip(*n_words_lists))
    n_word_string = list(map(lambda n_word_list : " ".join(n_word_list), zip_n_words))

    zip_n_flags = list(zip(*n_words_flags))
    combine_flags = list(map(lambda flag_list : reduce(lambda x, y: x | y, flag_list), zip_n_flags))
    
    if inplace == True:
        df[out_words_col] = n_word_string
        df[out_flags_col] = combine_flags
    return n_word_string, combine_flags

In [10]:
tqdm.pandas()

In [11]:
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        #print(input_str)
        return np.NaN

In [12]:
def difflib_closest_match_score(input_str, match_str):
    if isinstance(match_str, str):
        input_str = input_str.replace(' ', '')
        match_str = match_str.replace(' ', '')
        score = difflib.SequenceMatcher(None, input_str.lower(), match_str.lower()).ratio()
        return score
    else:
        return np.NaN

In [13]:
# for n in range(1, 5):
#     word_group_col_name = f"{n}_words"
#     match_col_name = f"{n}_words_match"
#     match_score_col_name = f"{n}_words_match_score"
#     print(f"finding best match for {word_group_col_name}")
#     get_n_words_flagged(vol1_word_df, n)
#     vol1_word_df[match_col_name] = vol1_word_df[word_group_col_name].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))
#     vol1_word_df[match_score_col_name] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r[word_group_col_name], r[match_col_name]), axis = 1)

vol1_word_df = pd.read_pickle("../input/desc_box_df/vol1_entry_df.pkl").loc[:,['vol_num', 'page_num', 'block_num', 'block_num_absolute', 'block_bbox',
                                                                               'line_num', 'line_wmode', 'line_dir', 'line_bbox', 'span_num',
                                                                               'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender',
                                                                               'span_descender', 'span_origin', 'span_bbox', 'word_num', 'word',
                                                                               'word_bbox', 'pruned_word', 'pruned_word_bbox', '1_words', '1_flags',
                                                                               '1_words_match', '1_words_match_score', '2_words', '2_flags',
                                                                               '2_words_match', '2_words_match_score', '3_words', '3_flags',
                                                                               '3_words_match', '3_words_match_score', '4_words', '4_flags',
                                                                               '4_words_match', '4_words_match_score']]

In [14]:
# things to add: 
# make sure they're on the same line (fixing the shift situation by doing it on group by?)
# make sure single word match is nothing completely unreasonable (e.g > 0.1 or something like that)
# make sure no words in the grouping are italics 
likely_results = vol1_word_df[((~(vol1_word_df['1_flags'].apply(is_italic)) & 
                                (vol1_word_df['1_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['2_flags'].apply(is_italic)) & 
                                (vol1_word_df['2_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['3_flags'].apply(is_italic)) & 
                                (vol1_word_df['3_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['1_flags'].apply(is_italic)) & 
                                (vol1_word_df['1_words_match_score'] > 0.85))) &
                              (vol1_word_df['page_num'] < 616)]

In [15]:
vol1_word_df['line_id'] = vol1_word_df.progress_apply(lambda r : (r['page_num'], r['block_num'], r['line_num']), axis = 1) 

is_page_title = vol1_word_df.groupby('line_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())
vol1_word_df['is_page_title'] = is_page_title

title_df = vol1_char_df.loc[vol1_char_df.groupby(['page_num', 'block_num', 'line_num'])['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())]
title_char_height = title_df.groupby(['page_num', 'block_num', 'line_num'])['char_bbox'].transform(lambda x: x.apply(lambda y: y[3] - y[1])).mean()

def get_page_title_mean_y(row):
    if row['is_page_title'] == True:
        return (row['line_bbox'][1] + row['line_bbox'][3])/2
    else:
        return np.nan
vol1_word_df['page_title_mean_y'] = vol1_word_df.progress_apply(get_page_title_mean_y, axis = 1)

num_pages = vol1_word_df['page_num'].max() + 1
for page_num in tqdm(range(num_pages)):
    vol1_word_df.loc[vol1_word_df['page_num'] == page_num, 'page_title_mean_y'] = vol1_word_df.loc[(vol1_word_df['page_num'] == page_num) & (vol1_word_df['is_page_title'] == True), 'page_title_mean_y'].mean()


def is_title_line(row):
    if abs(row['page_title_mean_y'] - ((row['line_bbox'][1] + row['line_bbox'][3])/2)) < (title_char_height/2):
        return True
    else:
        return False
vol1_word_df['is_title_line'] = vol1_word_df.progress_apply(is_title_line, axis = 1)

vol1_word_df = vol1_word_df[vol1_word_df['is_title_line'] == False].copy()

100%|██████████| 316313/316313 [00:01<00:00, 178519.12it/s]


In [16]:
from fitz.utils import getColor

In [17]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [18]:
#possible_stops = vol1_word_df[((vol1_word_df['word'].str.isupper()) & (vol1_word_df['pruned_word'].apply(len) > 2)) | (vol1_word_df['word'].isin(vol1_genera))]
genus_parts = vol1_word_df[(vol1_word_df['word'].isin(vol1_genera))] #or technically doing a close match etc 
middle_uppers = vol1_word_df[(vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & (vol1_word_df['word'].str.isupper())]
possible_stops = vol1_word_df[((vol1_word_df['word'].isin(vol1_genera))) | ((vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & (vol1_word_df['word'].str.isupper()) & (vol1_word_df['pruned_word'].apply(len) > 2))]

In [19]:
is_binomial = ((~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['2_flags'].apply(is_italic)) & (vol1_word_df['2_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['3_flags'].apply(is_italic)) & (vol1_word_df['3_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85))) 

is_stop = (((vol1_word_df['word'].isin(vol1_genera))) | 
           ((vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & 
            (vol1_word_df['word'].str.isupper()) & 
            (vol1_word_df['pruned_word'].apply(len) > 2)))

break_page_num = vol1_word_df[(is_binomial) | (is_stop)]['page_num']
break_block_num = vol1_word_df[(is_binomial) | (is_stop)]['block_num']
break_line_num = vol1_word_df[(is_binomial) | (is_stop)]['line_num']
break_id = list(zip(break_page_num, break_block_num, break_line_num))
vol1_word_df['section_break'] = vol1_word_df['line_id'].isin(break_id)

In [20]:
def get_section_id(row):
    if row['section_break'] == True:
        return row['line_id']
    else:
        return np.nan
vol1_word_df['section_id'] = vol1_word_df.progress_apply(get_section_id, axis = 1)

100%|██████████| 314517/314517 [00:01<00:00, 299294.27it/s]


In [21]:
vol1_word_df['section_id'].ffill(inplace=True)

In [22]:
def get_section_start_y(row):
    if row['section_break'] == True:
        return row['line_bbox'][1]
    else:
        return np.nan
# ssection break start y ... making sure we didn't "go up" in ffil
vol1_word_df['section_start_y'] = vol1_word_df.progress_apply(get_section_start_y, axis = 1)
vol1_word_df['section_start_y'].ffill(inplace=True)

100%|██████████| 314517/314517 [00:01<00:00, 278580.17it/s]


In [23]:
def get_section_start_y(row):
    if row['section_break'] == True:
        return row['line_bbox'][1]
    else:
        return np.nan
# ssection break start y ... making sure we didn't "go up" in ffil
vol1_word_df['section_start_y'] = vol1_word_df.progress_apply(get_section_start_y, axis = 1)
vol1_word_df['section_start_y'].ffill(inplace=True)

100%|██████████| 314517/314517 [00:01<00:00, 288676.41it/s]


In [24]:
# break down line coords
vol1_word_df['line_x0'] = vol1_word_df["line_bbox"].apply(lambda x: x[0])
vol1_word_df['line_y0'] = vol1_word_df["line_bbox"].apply(lambda x: x[1])
vol1_word_df['line_x1'] = vol1_word_df["line_bbox"].apply(lambda x: x[2])
vol1_word_df['line_y1'] = vol1_word_df["line_bbox"].apply(lambda x: x[3])

#sections_coords: 
vol1_word_df["section_x0"] = vol1_word_df.groupby(['page_num', 'section_id'])['line_x0'].transform('min')
vol1_word_df["section_y0"] = vol1_word_df.groupby(['page_num', 'section_id'])['line_y0'].transform('min')
#vol1_word_df["section_y0"] = vol1_word_df[['section_y0_all','section_y0_all']].max(axis=1)
vol1_word_df["section_x1"] = vol1_word_df.groupby(['page_num', 'section_id'])['line_x1'].transform('max')
vol1_word_df["section_y1"] = vol1_word_df.groupby(['page_num', 'section_id'])['line_y1'].transform('max')

#section_bbox:
vol1_word_df["section_bbox"] = vol1_word_df.apply(lambda r: (r["section_x0"], r["section_y0"], r["section_x1"], r["section_y1"]), axis = 1)

#drop extra cols:
vol1_word_df.drop(columns= ["line_x0", "line_y0", "line_x1", "line_y1", "section_x0", "section_y0", "section_x1", "section_y1"], inplace = True)

In [25]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [26]:
# section_groups = vol1_word_df.groupby('section_id')
# for name, section in tqdm(section_groups):
#     #for each group in the section ids I need to do the following:
#     page_num = int(vol1_word_df.loc[i, 'page_num'])
#     # block_bbox = vol1_word_df.loc[i, 'block_bbox']
#     # block_bbox = vol1_word_df.loc[i, 'block_bbox']
#     section_bbox = vol1_word_df.loc[i, 'section_bbox']
#     page = vol1_doc[page_num]
#     r_box = fitz.Rect(section_bbox)
#     #annot_high = page.add_highlight_annot(r_high)
#     #annot_high.set_colors(stroke=getColor("antiquewhite"))
#     annot_rect = page.add_rect_annot(r_box)
#     #annot_high.update()
#     annot_rect.update()


# marked_epithet_fname = "../output/local/main_text/sections_v1.pdf"
# vol1_doc.save(marked_epithet_fname)

In [27]:
# vol1_doc = fitz.open(vol1_path)
# vol2_doc = fitz.open(vol2_path)
# vol3_doc = fitz.open(vol3_path)

# section_groups = vol1_word_df.groupby('section_id')
# for name, section in tqdm(section_groups):
#     page_num = int(name[0])
#     page = vol1_doc[page_num]
#     section_bbox = section.iloc[0]['section_bbox']
#     r_box = fitz.Rect(section_bbox)
#     annot_rect = page.add_rect_annot(r_box)
#     annot_rect.update()

# marked_epithet_fname = "../output/local/main_text/sections_v3.pdf"
# vol1_doc.save(marked_epithet_fname)

In [28]:
binom_page_num = vol1_word_df[(is_binomial)]['page_num']
binom_block_num = vol1_word_df[(is_binomial)]['block_num']
binom_line_num = vol1_word_df[(is_binomial)]['line_num']
binom_id = list(zip(binom_page_num, binom_block_num, binom_line_num))
vol1_word_df['binom_section'] = vol1_word_df['line_id'].isin(binom_id)

In [29]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

for page_num in tqdm(range(num_pages)):
    section_groups = vol1_word_df[vol1_word_df['page_num'] == page_num].groupby('section_id')
    for name, section in section_groups:
        page = vol1_doc[page_num]
        section_id = section.iloc[0]['section_id']
        section_bbox = section.iloc[0]['section_bbox']
        if section_id in binom_id:
            r_box = fitz.Rect(section_bbox)
            annot_rect = page.add_rect_annot(r_box)
            annot_rect.update()

marked_epithet_fname = "../output/local/main_text/binom_sections_new_page_cont_v4.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 642/642 [00:00<00:00, 798.87it/s]


In [40]:
print("\n".join(vol1_word_df[(vol1_word_df['page_num'] == 174) & (vol1_word_df['section_id'] == (174, 1, 0)) & (vol1_word_df['word_num'] == 0)]['word']))

Cutandia
Festuca
(PL
.
ses
ture
non
Floraison:
S.
Aire
GYNOSORUS
les
latéralement,
membraneuses
membraneuse,
Paléa
étroites,
Panicule
:
C.
Panicule
'
Arêtes
;
C.
j
Arêtes...
:
C.
s.


In [38]:
vol1_word_df.to_pickle("../input/desc_box_df/vol1_desc_df_v2.pkl")

In [30]:
vol1_word_df[vol1_word_df['page_num'] == 96][['word', 'section_id']]

,word,section_id
253975,p.,"(95, 15, 0)"
253977,326,"(95, 15, 0)"
253980,de,"(95, 15, 0)"
253982,sa,"(95, 15, 0)"
253984,Flore,"(95, 15, 0)"
...,...,...
256877,"Turquie,","(96, 27, 0)"
256885,Akher,"(96, 27, 0)"
256890,"Dagh,","(96, 27, 0)"
256895,"Amanus,","(96, 27, 0)"


In [225]:
# and this is pretty great :)

In [240]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/binom_sections_page_cont_v1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 3505.32it/s]


In [ ]:
# prev cont -- not tested

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
prev_cont = True
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)) or (cont and prev_cont):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    if cont or prev_cont:
        prev_cont = True
    else: 
        prev_cont = False
    prev_sec = section_id

In [241]:
vol1_word_df.to_pickle("../input/desc_box_df/vol1_desc_df.pkl")

In [242]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)
vol1_doc = fitz.open("/Users/shidedehghani/Library/Mobile Documents/com~apple~CloudDocs/Research/rLAB/Maryam's_repo/Aaron/results-book-1.pdf")
section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/arronXme_vol1_v1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 2718.45it/s]


In [254]:
cont = vol1_word_df.groupby('section_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())

0            NaN
6            NaN
14           NaN
16           NaN
20           NaN
           ...  
1761774    False
1761781    False
1761784    False
1761788    False
1761792    False
Name: word, Length: 316313, dtype: object

In [276]:
cont = vol1_word_df.groupby('section_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())

In [277]:
count = vol1_word_df.groupby(['page_num', 'section_id'])['section_id'].transform('count')

In [278]:
vol1_word_df[(cont ==  True) & (count == 1)]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,4_words_match,4_words_match_score,line_id,section_break,section_id,section_bbox,section_start_y,section_y0_abs,section_y0_all,binom_section
21900,1,11,0,0,"(74.16000366210938, 38.59002685546875, 382.088...",0,0,"(1.0, 0.0)","(74.16000366210938, 38.59002685546875, 96.4082...",0,...,NaN,NaN,"(11, 0, 0)",False,"(10, 0, 0)","(28.799999237060547, 60.71002197265625, 551.89...",60.710022,38.590027,38.590027,False
100537,1,40,0,0,"(34.560001373291016, 28.765003204345703, 265.6...",0,0,"(1.0, 0.0)","(34.560001373291016, 28.765003204345703, 56.72...",0,...,NaN,NaN,"(40, 0, 0)",False,"(38, 0, 1)","(19.68000030517578, 27.30499267578125, 430.985...",27.304993,27.304993,27.304993,False
528224,1,197,0,0,"(50.400001525878906, 20.6199951171875, 278.865...",0,0,"(1.0, 0.0)","(50.400001525878906, 20.6199951171875, 67.4081...",0,...,NaN,NaN,"(197, 0, 0)",False,"(196, 0, 0)","(10.800000190734863, 23.3699951171875, 398.037...",23.369995,20.619995,20.619995,False
603711,1,223,0,0,"(41.52000045776367, 24.05999755859375, 271.665...",0,0,"(1.0, 0.0)","(41.52000045776367, 24.05999755859375, 58.8647...",0,...,NaN,NaN,"(223, 0, 0)",False,"(222, 0, 0)","(26.8799991607666, 13.969970703125, 399.190093...",13.969971,13.969971,13.969971,False
642190,1,237,0,0,"(37.68000030517578, 22.5, 267.8266906738281, 3...",0,0,"(1.0, 0.0)","(37.68000030517578, 22.5, 55.00499725341797, 3...",0,...,NaN,NaN,"(237, 0, 0)",False,"(236, 0, 0)","(18.959999084472656, 27.8900146484375, 403.025...",27.890015,22.500000,22.500000,False
681005,1,251,0,0,"(33.36000061035156, 25.29998779296875, 263.505...",0,0,"(1.0, 0.0)","(33.36000061035156, 25.29998779296875, 50.5206...",0,...,NaN,NaN,"(251, 0, 0)",False,"(250, 0, 0)","(32.400001525878906, 24.6099853515625, 400.597...",24.609985,24.609985,24.609985,False
799512,1,293,0,0,"(33.84000015258789, 22.405017852783203, 265.16...",0,0,"(1.0, 0.0)","(33.84000015258789, 22.405017852783203, 50.859...",0,...,NaN,NaN,"(293, 0, 0)",False,"(292, 0, 0)","(26.8799991607666, 26.085010528564453, 411.744...",26.085011,22.405018,22.405018,False
928405,1,337,0,0,"(39.599998474121094, 25.9000244140625, 270.705...",0,0,"(1.0, 0.0)","(39.599998474121094, 25.9000244140625, 57.5585...",0,...,NaN,NaN,"(337, 0, 0)",False,"(336, 0, 0)","(27.600000381469727, 22.010009765625, 397.2085...",22.010010,22.010010,22.010010,False
932366,1,339,0,0,"(46.79999923706055, 26.8599853515625, 278.6283...",0,0,"(1.0, 0.0)","(46.79999923706055, 26.8599853515625, 64.97999...",0,...,NaN,NaN,"(339, 0, 0)",False,"(338, 0, 0)","(46.79999923706055, 19.40997314453125, 385.322...",19.409973,19.409973,19.409973,False
1064598,1,385,0,0,"(42.720001220703125, 27.634973526000977, 274.2...",0,0,"(1.0, 0.0)","(42.720001220703125, 27.634973526000977, 59.96...",0,...,NaN,NaN,"(385, 0, 0)",False,"(384, 0, 0)","(22.559999465942383, 20.28997802734375, 372.96...",20.289978,20.289978,20.289978,False


In [ ]:
(cont == True) 

### putting the boxes on different pages together and correctly picking up multipage entries

In [ ]:
def possible_cont_section(row):
    if row['count'] == 1 and row['binom_section'] == False: #won't work -- sad 
        return np.NaN
    if row['cont']:
        return True
    if row['count'] > 1:
        return False

vol1_word_df['section_cont'] = vol1_word_df.progress_apply(get_section_start_y, axis = 1)
vol1_word_df['section_cont'].ffill(inplace=True)

In [279]:
vol1_word_df

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,4_words_match,4_words_match_score,line_id,section_break,section_id,section_bbox,section_start_y,section_y0_abs,section_y0_all,binom_section
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False
6,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False
14,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False
16,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False
20,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761774,1,641,4,6,"(154.0800018310547, 284.6300048828125, 227.289...",0,0,"(1.0, 0.0)","(154.0800018310547, 284.6300048828125, 227.289...",0,...,NaN,NaN,"(641, 4, 0)",True,"(641, 4, 0)","(154.0800018310547, 284.6300048828125, 227.289...",284.630005,284.630005,284.630005,False
1761781,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False
1761784,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False
1761788,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False


In [ ]:
vol1_word_df.groupby(['page_num', 'section_id'])['section_id'].transform('count') 

### other vols 

In [280]:
vol1_entry_df = pd.read_pickle("../input/desc_box_df/vol1_entry_df.pkl")
vol2_entry_df = pd.read_pickle("../input/desc_box_df/vol2_entry_df.pkl")
vol3_entry_df = pd.read_pickle("../input/desc_box_df/vol3_entry_df.pkl")

In [284]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)
i = 1
for vol_word_df, vol_doc in [(vol1_entry_df, vol1_doc), (vol2_entry_df, vol2_doc), (vol3_entry_df, vol3_doc)]:

    is_binomial = ((~(vol_word_df['1_flags'].apply(is_italic)) & (vol_word_df['1_words_match_score'] > 0.85)) | 
               (~(vol_word_df['2_flags'].apply(is_italic)) & (vol_word_df['2_words_match_score'] > 0.85)) | 
               (~(vol_word_df['3_flags'].apply(is_italic)) & (vol_word_df['3_words_match_score'] > 0.85)) | 
               (~(vol_word_df['1_flags'].apply(is_italic)) & (vol_word_df['1_words_match_score'] > 0.85))) 
    binom_page_num = vol_word_df[(is_binomial)]['page_num']
    binom_block_num = vol_word_df[(is_binomial)]['block_num']
    binom_line_num = vol_word_df[(is_binomial)]['line_num']
    binom_id = list(zip(binom_page_num, binom_block_num, binom_line_num))

    section_groups = vol_word_df.groupby('section_id')
    prev_sec = None
    for name, section in tqdm(section_groups):
        cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
        page_num = int(name[0])
        page = vol_doc[page_num]
        section_id = section.iloc[0]['section_id']
        section_bbox = section.iloc[0]['section_bbox']
        if section_id in binom_id or (cont and (prev_sec in binom_id)):
            r_box = fitz.Rect(section_bbox)
            annot_rect = page.add_rect_annot(r_box)
            annot_rect.update()
        prev_sec = section_id
    marked_epithet_fname = f"../output/local/main_text/entry_boxes_vol{i}.pdf"
    vol_doc.save(marked_epithet_fname)
    i += 1 

100%|██████████| 2095/2095 [00:00<00:00, 3374.28it/s]


### page continueing try 2 

In [334]:
vol1_word_df

/opt/homebrew/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/opt/homebrew/lib/python3.10/site-packages/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,line_id,section_break,section_id,section_bbox,section_start_y,section_y0_abs,section_y0_all,binom_section,section_cont,section_page_count
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False,NaN,0
6,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False,NaN,0
14,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False,NaN,0
16,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False,NaN,0
20,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(0, 0, 0)",False,NaN,"(nan, nan, nan, nan)",NaN,NaN,NaN,False,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761774,1,641,4,6,"(154.0800018310547, 284.6300048828125, 227.289...",0,0,"(1.0, 0.0)","(154.0800018310547, 284.6300048828125, 227.289...",0,...,"(641, 4, 0)",True,"(641, 4, 0)","(154.0800018310547, 284.6300048828125, 227.289...",284.630005,284.630005,284.630005,False,False,6
1761781,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False,False,6
1761784,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False,False,6
1761788,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(641, 5, 0)",True,"(641, 5, 0)","(134.63999938964844, 294.22998046875, 245.3221...",294.229980,294.229980,294.229980,False,False,6


In [326]:
cont = vol1_word_df.groupby('section_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())
#count = vol1_word_df.groupby(['page_num', 'section_id'])#['section_id'].transform(lambda x : x.groupby('section_id').ngroups)
count = vol1_word_df.groupby('page_num')['section_id'].transform('nunique')

In [ ]:
df['C'] = vol1_word_df.loc[vol1_word_df['section_break'] == True].groupby('A')['B'].transform('nunique')

# fill NaN values with 1 for groups that do not meet the condition ... for a case where only non break values exist?
df['C'].fillna(1, inplace=True)

In [327]:
vol1_word_df['section_cont'] = cont
vol1_word_df['section_page_count'] = count

In [328]:
vol1_word_df[vol1_word_df['section_page_count'] <= 1]['page_num'].unique()

array([  0,   2,   3,   7,   9,  10,  12,  39,  45,  47,  48,  51,  76,
       196, 222, 236, 250, 292, 336, 338, 384, 430, 438, 474, 540, 560,
       562, 592, 598, 600, 602, 607, 609, 614, 639])

In [329]:
is_binomial = ((~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['2_flags'].apply(is_italic)) & (vol1_word_df['2_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['3_flags'].apply(is_italic)) & (vol1_word_df['3_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85))) 

is_stop = (((vol1_word_df['word'].isin(vol1_genera))) | 
           ((vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & 
            (vol1_word_df['word'].str.isupper()) & 
            (vol1_word_df['pruned_word'].apply(len) > 2)))

break_page_num = vol1_word_df[(is_binomial) | (is_stop)]['page_num']
break_block_num = vol1_word_df[(is_binomial) | (is_stop)]['block_num']
break_line_num = vol1_word_df[(is_binomial) | (is_stop)]['line_num']
break_id = list(zip(break_page_num, break_block_num, break_line_num))
vol1_word_df['section_break'] = vol1_word_df['line_id'].isin(break_id)

def get_section_id(row):
    if row['section_break'] == True:
        return row['line_id']
    else:
        return np.nan
vol1_word_df['section_id'] = vol1_word_df.progress_apply(get_section_id, axis = 1)
vol1_word_df['section_id'].ffill(inplace=True)

100%|██████████| 316313/316313 [00:01<00:00, 278995.91it/s]


In [333]:
vol1_word_df[vol1_word_df['page_num'] == 263]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,line_id,section_break,section_id,section_bbox,section_start_y,section_y0_abs,section_y0_all,binom_section,section_cont,section_page_count
716476,1,263,0,0,"(34.79999923706055, 22.804981231689453, 264.67...",0,0,"(1.0, 0.0)","(34.79999923706055, 22.804981231689453, 51.191...",0,...,"(263, 0, 0)",False,"(262, 10, 0)","(18.479999542236328, 235.8699951171875, 405.06...",235.869995,22.804981,22.804981,False,False,2
716479,1,263,0,0,"(34.79999923706055, 22.804981231689453, 264.67...",1,0,"(1.0, 0.0)","(84.47938537597656, 22.804981231689453, 89.765...",0,...,"(263, 0, 1)",False,"(262, 10, 0)","(18.479999542236328, 235.8699951171875, 405.06...",235.869995,22.804981,22.804981,False,False,2
716480,1,263,0,0,"(34.79999923706055, 22.804981231689453, 264.67...",2,0,"(1.0, 0.0)","(187.2868194580078, 22.804981231689453, 264.67...",0,...,"(263, 0, 2)",True,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
716488,1,263,0,0,"(34.79999923706055, 22.804981231689453, 264.67...",2,0,"(1.0, 0.0)","(187.2868194580078, 22.804981231689453, 264.67...",0,...,"(263, 0, 2)",True,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
716493,1,263,1,1,"(54.959999084472656, 45.71002197265625, 418.22...",0,0,"(1.0, 0.0)","(54.959999084472656, 45.71002197265625, 418.22...",0,...,"(263, 1, 0)",False,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719037,1,263,26,29,"(32.880001068115234, 568.6699829101562, 417.78...",2,0,"(1.0, 0.0)","(32.880001068115234, 592.1900024414062, 417.33...",0,...,"(263, 26, 2)",False,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
719039,1,263,26,29,"(32.880001068115234, 568.6699829101562, 417.78...",2,0,"(1.0, 0.0)","(32.880001068115234, 592.1900024414062, 417.33...",0,...,"(263, 26, 2)",False,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
719049,1,263,26,29,"(32.880001068115234, 568.6699829101562, 417.78...",2,0,"(1.0, 0.0)","(32.880001068115234, 592.1900024414062, 417.33...",0,...,"(263, 26, 2)",False,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2
719052,1,263,26,29,"(32.880001068115234, 568.6699829101562, 417.78...",2,0,"(1.0, 0.0)","(32.880001068115234, 592.1900024414062, 417.33...",0,...,"(263, 26, 2)",False,"(263, 0, 2)","(31.920000076293945, 22.804981231689453, 418.2...",22.804981,22.804981,22.804981,False,True,2


In [336]:
vol1_word_df[vol1_word_df['page_num'] == 263]['line_id']

716476     (263, 0, 0)
716479     (263, 0, 1)
716480     (263, 0, 2)
716488     (263, 0, 2)
716493     (263, 1, 0)
              ...     
719037    (263, 26, 2)
719039    (263, 26, 2)
719049    (263, 26, 2)
719052    (263, 26, 2)
719057    (263, 26, 2)
Name: line_id, Length: 466, dtype: object

In [357]:
is_page_title = vol1_word_df.groupby('line_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())

In [ ]:
is_page_title = vol1_word_df.groupby('line_id')['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())
vol1_word_df['is_page_title'] = is_page_title

title_df = vol1_char_df.loc[vol1_char_df.groupby(['page_num', 'block_num', 'line_num'])['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())]
title_char_height = title_df.groupby(['page_num', 'block_num', 'line_num'])['char_bbox'].transform(lambda x: x.apply(lambda y: y[3] - y[1])).mean()


num_pages = vol1_word_df['page_num'].max() + 1
for page_num in tqdm(range(num_pages)):
    vol1_word_df.loc[vol1_word_df['page_num'] == page_num, 'page_title_mean_y'] = vol1_word_df.loc[(vol1_word_df['page_num'] == page_num) & (vol1_word_df['is_page_title'] == True), 'page_title_mean_y'].mean()


def is_title_line(row):
    if abs(row['page_title_mean_y'] - ((row['line_bbox'][1] + row['line_bbox'][3])/2)) < (title_char_height/2):
        return True
    else:
        return False
vol1_word_df['is_title_line'] = vol1_word_df.progress_apply(is_title_line, axis = 1)

vol1_word_df = vol1_word_df[vol1_word_df['is_title_line'] == False]

In [358]:
vol1_word_df['is_page_title'] = is_page_title

In [359]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)
section_groups = vol1_word_df.groupby('line_id')
prev_sec = None
for name, section in tqdm(section_groups):
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['line_id']
    section_bbox = section.iloc[0]['line_bbox']
    is_title = section.iloc[0]['is_page_title']
    if is_title:
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/page_titleNoNum_vol1_v1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 36974/36974 [00:04<00:00, 8005.52it/s]


In [352]:
vol1_word_df[(vol1_word_df['is_page_title'] == True) & (vol1_word_df['page_num'] > 200)].loc[564926]

vol_num                                                                1
page_num                                                             209
block_num                                                              0
block_num_absolute                                                     0
block_bbox             (37.68000030517578, 31.010009765625, 269.49542...
line_num                                                               0
line_wmode                                                             0
line_dir                                                      (1.0, 0.0)
line_bbox              (37.68000030517578, 31.010009765625, 54.568199...
span_num                                                               0
span_size                                                       9.222256
span_flags                                                             4
span_font                                                    Times-Roman
span_color                                         

In [ ]:
#check for NOUVELLE FLORE ONLY and then mark any thing with the same y value +- error as title line 

In [355]:
vol1_char_df['char_bbox'].apply(lambda x : x[3] - x[1]).mean()

10.585539172451485

In [377]:
title_df = vol1_char_df.loc[vol1_char_df.groupby(['page_num', 'block_num', 'line_num'])['word'].transform(lambda x : x.isin(['NOUVELLE', 'FLORE']).any())]
title_char_height = title_df.groupby(['page_num', 'block_num', 'line_num'])['char_bbox'].transform(lambda x: x.apply(lambda y: y[3] - y[1])).mean()

In [378]:
title_char_height

9.212780283902257

In [ ]:
vol1_word_df[vol1_word_df['is_page_title'] == True]

.groupby(['page_num', 'block_num', 'line_num'])['char_bbox'].transform(lambda x: x.apply(lambda y: y[3] - y[1])).mean()




In [384]:
def get_page_title_mean_y(row):
    if row['is_page_title'] == True:
        return (row['line_bbox'][1] + row['line_bbox'][3])/2
    else:
        return np.nan

num_pages = vol1_word_df['page_num'].max() + 1
for page_num in tqdm(range(num_pages)):
    vol1_word_df.loc[vol1_word_df['page_num'] == page_num, 'page_title_mean_y'] = vol1_word_df.loc[(vol1_word_df['page_num'] == page_num) & (vol1_word_df['is_page_title'] == True), 'page_title_mean_y'].mean()

100%|██████████| 642/642 [00:00<00:00, 842.97it/s]


In [385]:
def is_title_line(row):
    if abs(row['page_title_mean_y'] - ((row['line_bbox'][1] + row['line_bbox'][3])/2)) < (title_char_height/2):
        return True
    else:
        return False

vol1_word_df['is_title_line'] = vol1_word_df.progress_apply(is_title_line, axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 168036.20it/s]


In [386]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)
section_groups = vol1_word_df.groupby('line_id')
prev_sec = None
for i in tqdm(vol1_word_df.index):
    page_num = int(vol1_word_df.loc[i, 'page_num'])
    page = vol1_doc[page_num]
    line_id = vol1_word_df.loc[i, 'line_id']
    line_bbox = vol1_word_df.loc[i, 'line_bbox']
    is_title = vol1_word_df.loc[i, 'is_title_line']
    if is_title:
        r_box = fitz.Rect(line_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/is_title_line_vol1_v4.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 316313/316313 [00:07<00:00, 44469.38it/s]


In [387]:
vol1_word_df[vol1_word_df['page_num'] == 85]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,section_bbox,section_start_y,section_y0_abs,section_y0_all,binom_section,section_cont,section_page_count,is_page_title,page_title_mean_y,is_title_line
224476,1,85,0,0,"(36.959999084472656, 28.28997802734375, 269.07...",0,0,"(1.0, 0.0)","(36.959999084472656, 28.28997802734375, 43.777...",0,...,"(23.520000457763672, 573.4299926757812, 408.85...",573.429993,28.289978,28.289978,False,False,6,False,32.789978,True
224477,1,85,0,0,"(36.959999084472656, 28.28997802734375, 269.07...",1,0,"(1.0, 0.0)","(190.10287475585938, 28.28997802734375, 269.07...",0,...,"(190.10287475585938, 28.28997802734375, 269.07...",28.289978,28.289978,28.289978,False,True,6,True,32.789978,True
224485,1,85,0,0,"(36.959999084472656, 28.28997802734375, 269.07...",1,0,"(1.0, 0.0)","(190.10287475585938, 28.28997802734375, 269.07...",0,...,"(190.10287475585938, 28.28997802734375, 269.07...",28.289978,28.289978,28.289978,False,True,6,True,32.789978,True
224490,1,85,1,4,"(58.560001373291016, 51.420021057128906, 421.3...",0,0,"(1.0, 0.0)","(58.560001373291016, 51.420021057128906, 421.3...",0,...,"(35.279998779296875, 51.420021057128906, 421.3...",51.420021,51.420021,51.420021,True,False,6,False,32.789978,False
224498,1,85,1,4,"(58.560001373291016, 51.420021057128906, 421.3...",0,0,"(1.0, 0.0)","(58.560001373291016, 51.420021057128906, 421.3...",1,...,"(35.279998779296875, 51.420021057128906, 421.3...",51.420021,51.420021,51.420021,True,False,6,False,32.789978,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227238,1,85,27,62,"(34.560001373291016, 582.5400390625, 420.85964...",1,0,"(1.0, 0.0)","(34.560001373291016, 594.0599975585938, 420.85...",0,...,"(34.560001373291016, 465.0504150390625, 420.85...",465.050415,465.050415,465.050415,False,False,6,False,32.789978,False
227246,1,85,27,62,"(34.560001373291016, 582.5400390625, 420.85964...",1,0,"(1.0, 0.0)","(34.560001373291016, 594.0599975585938, 420.85...",0,...,"(34.560001373291016, 465.0504150390625, 420.85...",465.050415,465.050415,465.050415,False,False,6,False,32.789978,False
227249,1,85,27,62,"(34.560001373291016, 582.5400390625, 420.85964...",1,0,"(1.0, 0.0)","(34.560001373291016, 594.0599975585938, 420.85...",0,...,"(34.560001373291016, 465.0504150390625, 420.85...",465.050415,465.050415,465.050415,False,False,6,False,32.789978,False
227251,1,85,27,62,"(34.560001373291016, 582.5400390625, 420.85964...",1,0,"(1.0, 0.0)","(34.560001373291016, 594.0599975585938, 420.85...",0,...,"(34.560001373291016, 465.0504150390625, 420.85...",465.050415,465.050415,465.050415,False,False,6,False,32.789978,False
